In [ ]:
import sklearn
import sklearn.metrics
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import pandas
import mplhep
import pickle
import awkward
import awkward as ak
import glob
import bz2
import os
import tqdm
import fastjet
import vector
import uproot

mplhep.style.use("CMS")

import sys
sys.path += ["../../mlpf/"]

import jet_utils
sys.path += ["../../mlpf/plotting/"]

from plot_utils import ELEM_LABELS_CMS, ELEM_NAMES_CMS
from plot_utils import CLASS_LABELS_CMS, CLASS_NAMES_CMS
from plot_utils import cms_label, sample_label
from plot_utils import pid_to_text

import boost_histogram as bh

In [ ]:
# uproot.open("/local/joosep/mlpf/cms/20240823_simcluster/nopu/QCDForPF_14TeV_TuneCUETP8M1_cfi/root/pfntuple_1000000.root")["pfana/pftree"].keys()

In [ ]:
def load_tree(ttree):
    particles_pythia = ttree.arrays(["gen_pt", "gen_eta", "gen_phi", "gen_energy", "gen_pdgid", "gen_status", "gen_charge", "gen_daughters"])
    particles_cp = ttree.arrays(["caloparticle_pt", "caloparticle_eta", "caloparticle_phi", "caloparticle_energy", "caloparticle_pid", "caloparticle_charge"])
    particles_pf = ttree.arrays(["pfcandidate_pt", "pfcandidate_eta", "pfcandidate_phi", "pfcandidate_energy", "pfcandidate_pdgid"])
    genjet = ttree.arrays(["genjet_pt", "genjet_eta", "genjet_phi", "genjet_energy"])
    element = ttree.arrays(["element_eta", "element_type"])
    genmet = ttree.arrays(["genmet_pt"])
    return awkward.Array({"pythia": particles_pythia, "cp": particles_cp, "pf": particles_pf, "genjet": genjet, "genmet": genmet, "element": element})

def to_bh(data, bins, cumulative=False):
    h1 = bh.Histogram(bh.axis.Variable(bins))
    h1.fill(data)
    if cumulative:
        h1[:] = np.sum(h1.values()) - np.cumsum(h1)
    # h1[:] = sum_overflow_into_last_bin(h1.values(flow=True)[:])
    return h1


In [ ]:
rootfiles = list(glob.glob("/local/joosep/mlpf/cms//20250508_cmssw_15_0_5_d3c6d1/nopu/TTbar_14TeV_TuneCUETP8M1_cfi/root/pfntuple_8000*.root"))
#pklfiles = [r.replace("/root/", "/raw/").replace(".root", ".pkl.bz2") for r in rootfiles]

In [ ]:
#Download from https://jpata.web.cern.ch/jpata/mlpf/cms/20240823_simcluster/nopu/

tts = [
    load_tree(uproot.open(fn)["pfana/pftree"]) for fn in tqdm.tqdm(rootfiles)
]
tts = awkward.concatenate(tts, axis=0)

In [ ]:
# pickle_data = sum(
#     [pickle.load(bz2.BZ2File(fn)) for fn in pklfiles],
#     [],
# )

In [ ]:
# for i in range(len(pickle_data)):
#     for coll in ["ytarget", "ycand"]:
#         pickle_data[i][coll] = pandas.DataFrame(pickle_data[i][coll])
#         pickle_data[i][coll]["phi"] = np.arctan2(pickle_data[i][coll]["sin_phi"], pickle_data[i][coll]["cos_phi"])


# arrs_awk = {}

# for coll in ["ytarget", "ycand"]:
#     arrs_awk[coll] = {}
#     for feat in ["pid", "pt", "eta", "phi", "energy", "ispu"]:
#         arr = [np.array(p[coll][feat][p[coll]["pid"] != 0]) for p in pickle_data]
#         arrs_awk[coll][feat] = awkward.unflatten(awkward.concatenate(arr), [len(a) for a in arr])
        
# arrs_awk["pythia"] = {}
# arrs_awk["pythia"]["pid"] = awkward.from_regular([np.array(p["pythia"][:, 0]) for p in pickle_data])
# arrs_awk["pythia"]["pt"] = awkward.from_regular([np.array(p["pythia"][:, 1]) for p in pickle_data])
# arrs_awk["pythia"]["eta"] = awkward.from_regular([np.array(p["pythia"][:, 2]) for p in pickle_data])
# arrs_awk["pythia"]["phi"] = awkward.from_regular([np.array(p["pythia"][:, 3]) for p in pickle_data])
# arrs_awk["pythia"]["energy"] = awkward.from_regular([np.array(p["pythia"][:, 4]) for p in pickle_data])

In [ ]:
particles_pythia = tts["pythia"]
particles_cp = tts["cp"]
particles_pf = tts["pf"]

#genjets are directly from CMSSW, have some cuts applied on the particles
#see e.g. https://github.com/cms-sw/cmssw/blob/master/RecoJets/Configuration/python/GenJetParticles_cff.py#L8
#ideally, we should apply the same cuts on the pythia particles in mask_pythia
genjets = vector.awk(awkward.Array(
    awkward.zip(
        {   
            "pt": tts["genjet"]["genjet_pt"],
            "eta": tts["genjet"]["genjet_eta"],
            "phi": tts["genjet"]["genjet_phi"],
            "energy": tts["genjet"]["genjet_energy"],
        }
    ), with_name="Mometum4D"
))
genmet = tts["genmet"]["genmet_pt"]

In [ ]:
b = np.logspace(-3,3,100)
plt.figure(figsize=(5,5))

abs_pid = np.abs(particles_pythia["gen_pdgid"])
mask_pythia = (particles_pythia["gen_status"]==1) & (np.abs(particles_pythia["gen_eta"])<5)
mask_pythia_nonu = (particles_pythia["gen_status"]==1) & (np.abs(particles_pythia["gen_eta"])<5) & (abs_pid!=12) & (abs_pid!=14) & (abs_pid!=16)
mask_cp = np.abs(particles_cp["caloparticle_eta"])<5

plt.hist(awkward.flatten(particles_pythia[mask_pythia_nonu]["gen_pt"]), bins=b, label="Pythia", histtype="step")
plt.hist(awkward.flatten(particles_pf["pfcandidate_pt"]), bins=b, label="PF", histtype="step")
plt.hist(awkward.flatten(particles_cp[mask_cp]["caloparticle_pt"]), bins=b, label="CaloParticle", histtype="step")

plt.xscale("log")
plt.yscale("log")
plt.xlabel("Particle $p_T$ [GeV]")
plt.legend()

In [ ]:
b = np.linspace(-5,5,100)
plt.figure(figsize=(5,5))

plt.hist(awkward.flatten(particles_pythia[mask_pythia_nonu]["gen_eta"]), bins=b, label="Pythia", histtype="step")
plt.hist(awkward.flatten(particles_cp[mask_cp]["caloparticle_eta"]), bins=b, label="CaloParticle", histtype="step")
plt.hist(awkward.flatten(particles_pf["pfcandidate_eta"]), bins=b, label="PF", histtype="step")

plt.xlabel("Particle $\eta$")
plt.legend()

In [ ]:
def remap_pid(pdgid, charge):
    pid = np.abs(np.asarray(ak.flatten(pdgid)))
    pa = ak.flatten(np.abs(pdgid))
    pc = ak.flatten(np.abs(charge))
    pid[(pa!=11) & (pa!=13) & (pa!=22) & (pc==1)] = 211
    pid[(pa!=11) & (pa!=13) & (pa!=22) & (pc==0)] = 130
    return ak.unflatten(pid, ak.count(pdgid, axis=1))


In [ ]:
caloparticle_pid_remapped = remap_pid(particles_cp["caloparticle_pid"], particles_cp["caloparticle_charge"])
gen_pid_remapped = remap_pid(particles_pythia["gen_pdgid"], particles_pythia["gen_charge"])

In [ ]:
b = np.logspace(-3,4,100)

pid1 = np.abs(particles_pythia["gen_pdgid"])
pid2 = np.abs(particles_cp["caloparticle_pid"])
pid3 = np.abs(particles_pf["pfcandidate_pdgid"])

mask_pythia_nonu = (particles_pythia["gen_status"]==1) & (np.abs(particles_pythia["gen_eta"])<5) & (abs_pid!=12) & (abs_pid!=14) & (abs_pid!=16)
mask_cp = np.abs(particles_cp["caloparticle_eta"])<5

uniq_pid = np.unique(awkward.flatten(pid1[mask_pythia_nonu]))
fig, axs = plt.subplots(4,4, figsize=(20,20))

iax = 0
axs = axs.flatten()

for pid in uniq_pid:
    if (np.sum(pid1==pid)>0) & (np.sum(pid2==pid)>0):
        plt.sca(axs[iax])
        plt.hist(awkward.flatten(particles_pythia[mask_pythia_nonu & (pid1==pid)]["gen_pt"]), bins=b, label="Pythia", histtype="step")
        plt.hist(awkward.flatten(particles_cp[mask_cp & (pid2==pid)]["caloparticle_pt"]), bins=b, label="CaloParticle", histtype="step")
        plt.hist(awkward.flatten(particles_pf[(pid3==pid)]["pfcandidate_pt"]), bins=b, label="PF", histtype="step")
        
        plt.xscale("log")
        plt.yscale("log")
        plt.xlabel("Particle $p_T$ [GeV]")
        plt.legend(fontsize=8)
        plt.title("pdgid {}".format(pid))
        iax += 1
plt.tight_layout()

In [ ]:
b = np.logspace(-3,4,100)

pid1 = np.abs(gen_pid_remapped)
pid2 = np.abs(caloparticle_pid_remapped)
pid3 = np.abs(particles_pf["pfcandidate_pdgid"])

mask_pythia_nonu = (particles_pythia["gen_status"]==1) & (np.abs(particles_pythia["gen_eta"])<5) & (abs_pid!=12) & (abs_pid!=14) & (abs_pid!=16)
mask_cp = np.abs(particles_cp["caloparticle_eta"])<5

uniq_pid = np.unique(awkward.flatten(pid1[mask_pythia_nonu]))
fig, axs = plt.subplots(3,2, figsize=(2*5,3*5))

iax = 0
axs = axs.flatten()

for pid in uniq_pid:
    if (np.sum(pid1==pid)>0) & (np.sum(pid2==pid)>0):
        plt.sca(axs[iax])
        plt.hist(awkward.flatten(particles_pythia[mask_pythia_nonu & (pid1==pid)]["gen_pt"]), bins=b, label="Pythia", histtype="step")
        plt.hist(awkward.flatten(particles_cp[mask_cp & (pid2==pid)]["caloparticle_pt"]), bins=b, label="CaloParticle", histtype="step")
        plt.hist(awkward.flatten(particles_pf[(pid3==pid)]["pfcandidate_pt"]), bins=b, label="PF", histtype="step")
        
        plt.xscale("log")
        plt.yscale("log")
        plt.xlabel("Particle $p_T$ [GeV]")
        plt.legend(fontsize=8)
        plt.title("pid {}".format(pid))
        iax += 1
plt.tight_layout()

In [ ]:
#Sum pT in event
b = np.linspace(0,1000,100)
fig, axs = plt.subplots(1,2, figsize=(10,5))


plt.sca(axs[0])

plt.hist2d(
    awkward.to_numpy(awkward.sum(particles_pythia[mask_pythia_nonu]["gen_pt"], axis=1)),
    awkward.to_numpy(awkward.sum(particles_cp[mask_cp]["caloparticle_pt"], axis=1)),
    bins=(b,b), cmap="hot_r", #norm=matplotlib.colors.LogNorm()
)
plt.plot([0,2000],[0,2000], color="black", lw=0.5)
plt.xlabel("Pythia, no nu")
plt.ylabel("CaloParticle")

plt.sca(axs[1])

#it seems like caloparticles and p
pythia_ptcut = ((particles_pythia["gen_pdgid"]==22) & (particles_pythia["gen_pt"]>0.3)) | (particles_pythia["gen_pdgid"]!=22)

plt.title("$p_T>0.3$ GeV")
plt.hist2d(
    awkward.to_numpy(awkward.sum(particles_pythia[mask_pythia_nonu & pythia_ptcut]["gen_pt"], axis=1)),
    awkward.to_numpy(awkward.sum(particles_cp[mask_cp]["caloparticle_pt"], axis=1)),
    bins=(b,b), cmap="hot_r", #norm=matplotlib.colors.LogNorm()
)
plt.plot([0,2000],[0,2000], color="black", lw=0.5)
plt.xlabel("Pythia, no nu")
plt.ylabel("CaloParticle")
plt.tight_layout()
plt.suptitle("sum $p_T$")

## Jets

In [ ]:
jets_coll = {}
jets_coll["genjet"] = genjets
jet_constituents = {}

vec = vector.awk(
    awkward.zip(
        {   
            "pt": particles_pythia[mask_pythia_nonu]["gen_pt"],
            "eta": particles_pythia[mask_pythia_nonu]["gen_eta"],
            "phi": particles_pythia[mask_pythia_nonu]["gen_phi"],
            "energy": particles_pythia[mask_pythia_nonu]["gen_energy"],
        }
    )
)
jetdef = fastjet.JetDefinition(fastjet.antikt_algorithm, 0.4)
cluster = fastjet.ClusterSequence(vec.to_xyzt(), jetdef)
jets_coll["pythia_nonu"] = cluster.inclusive_jets(min_pt=3)
jet_constituents["pythia_nonu"] = cluster.constituent_index(min_pt=3)

vec = vector.awk(
    awkward.zip(
        {   
            "pt": particles_pythia[mask_pythia_nonu & pythia_ptcut]["gen_pt"],
            "eta": particles_pythia[mask_pythia_nonu & pythia_ptcut]["gen_eta"],
            "phi": particles_pythia[mask_pythia_nonu & pythia_ptcut]["gen_phi"],
            "energy": particles_pythia[mask_pythia_nonu & pythia_ptcut]["gen_energy"],
        }
    )
)
jetdef = fastjet.JetDefinition(fastjet.antikt_algorithm, 0.4)
cluster = fastjet.ClusterSequence(vec.to_xyzt(), jetdef)
jets_coll["pythia_nonu_ptcut"] = cluster.inclusive_jets(min_pt=3)
jet_constituents["pythia_nonu_ptcut"] = cluster.constituent_index(min_pt=3)

vec = vector.awk(
    awkward.zip(
        {   
            "pt": particles_pythia[mask_pythia]["gen_pt"],
            "eta": particles_pythia[mask_pythia]["gen_eta"],
            "phi": particles_pythia[mask_pythia]["gen_phi"],
            "energy": particles_pythia[mask_pythia]["gen_energy"],
        }
    )
)
jetdef = fastjet.JetDefinition(fastjet.antikt_algorithm, 0.4)
cluster = fastjet.ClusterSequence(vec.to_xyzt(), jetdef)
jets_coll["pythia"] = cluster.inclusive_jets(min_pt=3)

vec = vector.awk(
    awkward.zip(
        {   
            "pt": particles_cp[mask_cp]["caloparticle_pt"],
            "eta": particles_cp[mask_cp]["caloparticle_eta"],
            "phi": particles_cp[mask_cp]["caloparticle_phi"],
            "energy": particles_cp[mask_cp]["caloparticle_energy"],
        }
    )
)
jetdef = fastjet.JetDefinition(fastjet.antikt_algorithm, 0.4)
cluster = fastjet.ClusterSequence(vec.to_xyzt(), jetdef)
jets_coll["cp"] = cluster.inclusive_jets(min_pt=3)
jet_constituents["cp"] = cluster.constituent_index(min_pt=3)


vec = vector.awk(
    awkward.zip(
        {   
            "pt": particles_pf["pfcandidate_pt"],
            "eta": particles_pf["pfcandidate_eta"],
            "phi": particles_pf["pfcandidate_phi"],
            "energy": particles_pf["pfcandidate_energy"],
        }
    )
)
jetdef = fastjet.JetDefinition(fastjet.antikt_algorithm, 0.4)
cluster = fastjet.ClusterSequence(vec.to_xyzt(), jetdef)
jets_coll["pf"] = cluster.inclusive_jets(min_pt=3)
jet_constituents["pf"] = cluster.constituent_index(min_pt=3)

genjet_to_pythia = jet_utils.match_two_jet_collections(jets_coll, "genjet", "pythia", 0.1)
genjet_to_pythia_nonu_ptcut = jet_utils.match_two_jet_collections(jets_coll, "genjet", "pythia_nonu_ptcut", 0.1)
genjet_to_cp = jet_utils.match_two_jet_collections(jets_coll, "genjet", "cp", 0.1)
genjet_to_pf = jet_utils.match_two_jet_collections(jets_coll, "genjet", "pf", 0.1)

pythia_to_cp = jet_utils.match_two_jet_collections(jets_coll, "pythia", "cp", 0.1)
pythia_nonu_to_cp = jet_utils.match_two_jet_collections(jets_coll, "pythia_nonu", "cp", 0.1)
pythia_nonu_ptcut_to_cp = jet_utils.match_two_jet_collections(jets_coll, "pythia_nonu_ptcut", "cp", 0.1)

In [ ]:
b = np.logspace(0,3,200)
plt.hist(awkward.flatten(jets_coll["genjet"].pt), bins=b, histtype="step", lw=1, label="genjet")
plt.hist(awkward.flatten(jets_coll["cp"].pt), bins=b, histtype="step", lw=1, label="CaloParticle")
plt.hist(awkward.flatten(jets_coll["pythia"].pt), bins=b, histtype="step", lw=1, label="Pythia")
plt.hist(awkward.flatten(jets_coll["pythia_nonu"].pt), bins=b, histtype="step", lw=1, label="Pythia, NoNu")
plt.hist(awkward.flatten(jets_coll["pythia_nonu_ptcut"].pt), bins=b, histtype="step", lw=1, label="Pythia, NoNu, pt>0.3 GeV")
plt.xscale("log")
plt.yscale("log")
plt.xlabel("jet $p_T$ [GeV]")
plt.ylabel("number of jets")
plt.legend()

In [ ]:
plt.figure(figsize=(5,5))
b = np.logspace(-1,1,600)

# plt.hist(
#     awkward.flatten(
#         (jets_coll["pythia"][genjet_to_pythia["pythia"]].pt / jets_coll["genjet"][genjet_to_pythia["genjet"]].pt)
#     ), bins=b, histtype="step", lw=1, label="Pythia"
# );

plt.hist(
    awkward.flatten(
        (jets_coll["cp"][genjet_to_cp["cp"]].pt / jets_coll["genjet"][genjet_to_cp["genjet"]].pt)
    ), bins=b, histtype="step", lw=1, label="CaloParticle"
);

plt.hist(
    awkward.flatten(
        (jets_coll["pf"][genjet_to_pf["pf"]].pt / jets_coll["genjet"][genjet_to_pf["genjet"]].pt)
    ), bins=b, histtype="step", lw=1, label="PF"
);

plt.xscale("log")
plt.yscale("log")
plt.xlabel("jet $p_T$ / GenJet jet $p_T$")
plt.legend(loc=2, fontsize=12)
plt.axvline(1.0, color="black", ls="--", lw=0.5)

In [ ]:
plt.figure(figsize=(5,5))
b = np.linspace(0,2,600)

# plt.hist(
#     awkward.flatten(
#         (jets_coll["pythia"][genjet_to_pythia["pythia"]].pt / jets_coll["genjet"][genjet_to_pythia["genjet"]].pt)
#     ), bins=b, histtype="step", lw=1, label="Pythia"
# );

plt.hist(
    awkward.flatten(
        (jets_coll["cp"][genjet_to_cp["cp"]].pt / jets_coll["genjet"][genjet_to_cp["genjet"]].pt)
    ), bins=b, histtype="step", lw=1, label="CaloParticle"
);

plt.hist(
    awkward.flatten(
        (jets_coll["pf"][genjet_to_pf["pf"]].pt / jets_coll["genjet"][genjet_to_pf["genjet"]].pt)
    ), bins=b, histtype="step", lw=1, label="PF"
);

#plt.xscale("log")
#plt.yscale("log")
plt.xlabel("jet $p_T$ / GenJet jet $p_T$")
plt.legend(loc=2, fontsize=12)
plt.axvline(1.0, color="black", ls="--", lw=0.5)

## MET

In [ ]:
def met(pt, phi):
    px = pt * np.cos(phi)
    py = pt * np.sin(phi)
    pt = np.sqrt(awkward.sum(px, axis=1)**2 + awkward.sum(py, axis=1)**2)
    return pt

met_pythia_nonu = met(particles_pythia[mask_pythia_nonu]["gen_pt"], particles_pythia[mask_pythia_nonu]["gen_phi"])
met_pythia_nonu_ptcut = met(particles_pythia[mask_pythia_nonu & pythia_ptcut]["gen_pt"], particles_pythia[mask_pythia_nonu & pythia_ptcut]["gen_phi"])
met_cp = met(particles_cp[mask_cp]["caloparticle_pt"], particles_cp[mask_cp]["caloparticle_phi"])
met_pf = met(particles_pf["pfcandidate_pt"], particles_pf["pfcandidate_phi"])

In [ ]:
b = np.linspace(0,200,101)
plt.figure(figsize=(5,5))
plt.hist(genmet, bins=b, histtype="step", label="genMetTrue")
plt.hist(met_pythia_nonu, bins=b, histtype="step", label="Pythia, no nu")
plt.hist(met_cp, bins=b, histtype="step", label="CaloParticle")
plt.hist(met_pf, bins=b, histtype="step", label="PF")
#plt.xscale("log")
plt.yscale("log")
plt.legend(loc=1, fontsize=12)
plt.xlabel("MET [GeV]")

In [ ]:
b = np.linspace(0,5,101)
plt.figure(figsize=(5,5))
plt.plot([], [])
plt.hist((met_pythia_nonu/genmet[:, 0])[genmet[:, 0]>1], bins=b, histtype="step", label="Pythia, no nu")
plt.hist((met_cp/genmet[:, 0])[genmet[:, 0]>1], bins=b, histtype="step", label="CaloParticle")
plt.hist((met_pf/genmet[:, 0])[genmet[:, 0]>1], bins=b, histtype="step", label="PF")
#plt.xscale("log")
#plt.yscale("log")
plt.legend(loc=1, fontsize=12)
plt.title("genMET>1")
plt.xlabel("MET / genMET")

In [ ]:
plt.figure(figsize=(5,5))
fig, axs = plt.subplots(1,2, figsize=(10,5))

b = np.logspace(-2,3,100)
plt.sca(axs[0])
plt.hist2d(ak.to_numpy(genmet[:, 0]), ak.to_numpy(met_pythia_nonu), bins=(b,b), cmap="Blues")
plt.xlabel("genMetTrue")
plt.ylabel("Pythia MET")
plt.xscale("log")
plt.yscale("log")

plt.sca(axs[1])
plt.hist2d(ak.to_numpy(met_pythia_nonu), ak.to_numpy(met_cp), bins=(b,b), cmap="Blues")
plt.xlabel("Pythia MET")
plt.ylabel("CaloParticle MET")
plt.xscale("log")
plt.yscale("log")

plt.tight_layout()